# Face Recognition Using mlrun with OpenCV And PyTorch
 A complete pipeline of data processing, model training and serving function deployment.

### Declare global variables

In [ ]:
ARTIFACTS_PATH = '/User/demos/demos/faces/artifacts/'
CODE_PATH = '/User/demos/demos/faces/code/'

### Import and define mlrun functions for the pipeline 

In [ ]:
# nuclio: ignore
from mlrun import new_function, code_to_function, NewTask, mount_v3io
import kfp
from kfp import dsl

In [ ]:
encode_fn = code_to_function('encode-images', image='yjbds/mlrun-horovod-gpu:0.4.5', kind='job', filename=CODE_PATH + 'encode.py')
encode_fn.export(CODE_PATH + 'encode.yaml')
encode_fn.apply(mount_v3io())

In [ ]:
train_fn = new_function(name='train', image='yjbds/mlrun-horovod-gpu:0.4.5', kind='mpijob', command=CODE_PATH + 'train.py')
train_fn.export(CODE_PATH + 'train.yaml')
train_fn.spec.replicas = 2
train_fn.gpus(1)
train_fn.apply(mount_v3io())

In [ ]:
model_serving_function = code_to_function(
    name='recognize-faces', 
    filename='./nuclio-face-prediction.ipynb',
    kind='nuclio')

model_serving_function.apply(mount_v3io())

In [ ]:
api_serving_function = code_to_function(
    name='video-api-server', 
    filename='./nuclio-api-serving.ipynb',
    kind='nuclio')

api_serving_function.apply(mount_v3io())

In [ ]:
from mlrun import mlconf
mlconf.dbpath = 'http://mlrun-api:8080'

### Create pipeline

In [ ]:
@dsl.pipeline(
    name='face recognition pipeline',
    description='Creates and deploys a face recognition model'
)
def face_recognition_pipeline(with_cuda=True, with_horovod=True):
    
    encode = encode_fn.as_step(name='encode-images', handler='encode_images', out_path=ARTIFACTS_PATH, outputs=['idx2name', 'encodings_path'],
                       inputs={'cuda': with_cuda})
    
    train = train_fn.as_step(name='train', out_path=ARTIFACTS_PATH, outputs=['model'], 
                               inputs={'processed_data': encode.outputs['encodings_path'], 'cuda': with_cuda, 'horovod': with_horovod})
    
    deploy_model = model_serving_function.deploy_step(project='default', models={'face_rec_v1': train.outputs['model']})
    
    deploy_api = api_serving_function.deploy_step(project='default').after(deploy_model)
    

In [ ]:
client = kfp.Client(namespace='default-tenant')

In [ ]:
#For debug purposes compile pipeline code
kfp.compiler.Compiler().compile(face_recognition_pipeline, 'face_rec.yaml')

### Run pipeline

In [ ]:
arguments = {}
run_result = client.create_run_from_pipeline_func(face_recognition_pipeline, arguments=arguments, run_name='face_rec_1', experiment_name='face_rec')